# Map Processing Example
------------------

#### <font color="blue"> by: Jeremy Meinke (feel free to contact me here on GitHub, LinkedIn, or email)
------------------------------------------

This notebook will dive a bit more into how to process differently formatted maps, such as SPT and ACT.
It is a slight simplification of what has been conducted in our recent paper (ref to add).

The notebook is divided into two parts:
1.  Creating an SPT mask distance map - conveying distances (per pixel) to the nearest SPT mask pixel as used by Chown 2018.
2.  Steps to convert ACT maps into healpix format, with the option of subtracting a large-scale CMB Planck map (as done by our recent paper above).

---

In [1]:
from __future__ import division
import numpy as np
import pandas as pd
from astropy.table import Table
import healpy as hp
import matplotlib as mpl
import matplotlib.pyplot as plt
from pixell import enmap
from cycler import cycler
import time
import gc
###SPT maps are big... gc.collect() is a safety way to try to keep the notebook from using too much memory
import h5py

from stacking_code import stacking    ###My stacking code
from analysis_code import general     ###My general analysis code

###My personal matplotlib preferences
marker_cycle=["o","^","s","d","X","2","h","v",">"]
color_cycle=["k","b","r","c","orange","m","lime","grey","lavender"]
mpl.rcParams["axes.prop_cycle"] = cycler(color=color_cycle + color_cycle[::-1], marker=marker_cycle*2)

print("Last time run: %s"%time.asctime())

Last time run: Thu May 25 14:06:55 2023


**This next block will compile a set of .npy data files into a 3.2GB SPT Mask Distance healpix map for easy calling if you ever need to remove locations within a catalog if they are near one of these "holes" in the SPT-SZ map (see Chown et al. 2018 for details, they essentially cut a 5 arcmin hole around bright 150GHz pt sources and apodized the edges by a further 5 arcmin FWHM beam).**

*So this next block only needs to be run once, then you can just load the resultant healpix map whenever needed*

In [ ]:
## Comment this block out after first run through
## First time setup to generate the SPT-SZ mask distance map from smaller saved .npy files
divs = 20
spt_mask_distances = np.zeros(12*8192**2)  ## As the SPT HEALPix nside=8192
for d in range(divs):
    temp = np.load("./data/spt_mask_distances_map_subset%iof%i.npy"%(d+1, divs))
    ## Now assigning the distances in degrees (row 1) to their respective pixel index (row 0)
    spt_mask_distances[np.array(temp[0], dtype="int")] = temp[1]

## Now saving the map into healpix format:
hp.write_map("./data/spt_mask_distances_chown2018.fits", spt_mask_distances, nest=False, partial=True, dtype=np.float64, fits_IDL=False, coord="C", column_units="DEG")

In [ ]:
## Test load and plot of the result
hp.mollview(spt_mask_distances, coord="C", unit="Distance to SPT Mask [Deg]")
hp.graticule()
plt.show()